In [1]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np
import sys,os

In [2]:
import models
from models import weights_init

# Main code

In [3]:
#Libera as funcionalidades da biblioteca cudnn
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


In [4]:
nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64

In [5]:
!cd images && pwd


/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/images


## Setando as transformações

In [6]:
!ls images/images2/

ls: cannot access 'images/images2/': No such file or directory


In [7]:
!ls dataset

anime-faces  min_anime-faces  teste


In [8]:
dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
)

## Setando o Dataloader

In [9]:
batch_size=64
num_workers = 4

In [10]:

dataloader = tc.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=1)
#126 classes

## Setando o modelo

### Modelo DCGAN

In [11]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        main = nn.Sequential(
            # input is (nc) x 96 x 96
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), # 5,3,1 for 96x96
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
        )

        # Extra layers
        for t in range(n_extra_layers_d):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ndf * 8),
                            nn.Conv2d(ndf * 8, ndf * 8, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ndf * 8),
                            nn.BatchNorm2d(ndf * 8))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ndf * 8),
                            nn.LeakyReLU(0.2, inplace=True))


        main.add_module('final_layers.conv', nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False))
        main.add_module('final_layers.sigmoid', nn.Sigmoid())
        # state size. 1 x 1 x 1
        self.main = main

    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        return output.view(-1, 1)


In [12]:
# DCGAN model, fully convolutional architecture
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        #self.nz = nz
        #self.nc = nc
        #self.ngf = ngf
        main = nn.Sequential(
            # input is Z, going into a convolution
            # state size. nz x 1 x 1
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf) x 32 x 32
        )

        # Extra layers
        for t in range(n_extra_layers_g):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ngf),
                            nn.Conv2d(ngf, ngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ngf),
                            nn.BatchNorm2d(ngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ngf),
                            nn.LeakyReLU(0.2, inplace=True))

        main.add_module('final_layer.deconv', 
        	             nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)) # 5,3,1 for 96x96
        main.add_module('final_layer.tanh', 
        	             nn.Tanh())
            # state size. (nc) x 96 x 96

        self.main = main


    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        return nn.parallel.data_parallel(self.main, input, gpu_ids), 0

In [13]:
#netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)
netG_parallel = torch.nn.DataParallel(_netG_1(ngpu, nz, nc, ngf, n_extra_g))

In [14]:
#print(netG)
print(netG_parallel)

DataParallel (
  (module): _netG_1 (
    (main): Sequential (
      (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (2): LeakyReLU (0.2, inplace)
      (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (5): LeakyReLU (0.2, inplace)
      (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (8): LeakyReLU (0.2, inplace)
      (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (11): LeakyReLU (0.2, inplace)
      (extra-layers-0.64.conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (extra-layers-0.64.batchnorm):

In [15]:
#netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d)

netD_parallel = torch.nn.DataParallel(_netD_1(ngpu, nz, nc, ndf, n_extra_d))

In [16]:
print(netD_parallel)
#print(netD)

DataParallel (
  (module): _netD_1 (
    (main): Sequential (
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU (0.2, inplace)
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
      (4): LeakyReLU (0.2, inplace)
      (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (7): LeakyReLU (0.2, inplace)
      (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (10): LeakyReLU (0.2, inplace)
      (final_layers.conv): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (final_layers.sigmoid): Sigmoid ()
    )
  )
)


# Carregando pesos pré-treinados

In [17]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [18]:
#Parece ser um inicializador de pesos hardcoded
#netG.apply(weights_init)
netG_parallel.apply(weights_init)
#netD.apply(weights_init)
netD_parallel.apply(weights_init)
print(())

()


load=False
if load:
    netD.load_state_dict(tc.load('path_d'))
    
    netG.load_state_dict(torch.load('path_G'))


### Parametros de treinamento

In [19]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()


In [20]:

input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 64, 64])
torch.Size([64, 100, 1, 1])


In [21]:
#parser.add_argument('--binary', action='store_true', help='z from bernoulli distribution, with prob=0.5')
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
    bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
    fixed_noise = torch.bernoulli(bernoulli_prob)
else:
    fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)


In [22]:
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0


### Broadcast para CUDA, se quiser

In [23]:
if use_gpu:
    #netD.cuda()
    #netG.cuda()
    netD_parallel.cuda()
    netG_parallel.cuda()
    criterion = criterion.cuda()
    criterion_MSE = criterion_MSE.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

### Transformando tudo em variable

In [24]:

input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)


### Setando o optimizer

In [25]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
#optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerD = optim.Adam(netD_parallel.parameters(), lr = lr, betas = (beta1, beta2))

#optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG_parallel.parameters(), lr = lr, betas = (beta1, beta2))


### Criando o diretório vazio

In [26]:
outputDir = 'outputdir_train'

try:
    os.makedirs(outputDir)
except OSError as err:
    print("OS error: {0}".format(err))

## Treinando !

In [27]:
print(type(input))
print(input.size())
print(label.size())

<class 'torch.autograd.variable.Variable'>
torch.Size([64, 3, 64, 64])
torch.Size([64])


In [28]:
def train_both_networks(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir,
                        model_option =1,binary = False, epoch_interval = 1):
    use_gpu = tc.cuda.is_available()

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            start_iter = time.time()
            
            
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            # 1A - Train the detective network in the Real Dataset
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu, _ = data
            batch_size = real_cpu.size(0)
            input.data.resize_(real_cpu.size()).copy_(real_cpu)
            label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

            output = netD(input)
            errD_real = criterion(output, label)
            errD_real.backward()
            
            #######################################################
           
            #######################################################
            # 1B - Train the detective network in the False Dataset
            #######################################################
            
            D_x = output.data.mean()
            # train with fake
            noise.data.resize_(batch_size, nz, 1, 1)
            if binary:
                bernoulli_prob.resize_(noise.data.size())
                noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
            else:
                noise.data.normal_(0, 1)
            fake,z_prediction = netG(noise)
            label.data.fill_(fake_label)
            output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
            errD_fake = criterion(output, label)
            errD_fake.backward() # gradients for fake/real will be accumulated
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            optimizerD.step() # .step() can be called once the gradients are computed

            #######################################################

            #######################################################
            # (2) Update G network: maximize log(D(G(z)))
            #  Train the faker with de output from the Detective (but don't train the Detective)
            #############3#########################################
            netG.zero_grad()
            label.data.fill_(real_label) # fake labels are real for generator cost
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward(retain_variables=True) # True if backward through the graph for the second time
            if model_option == 2: # with z predictor
                errG_z = criterion_MSE(z_prediction, noise)
                errG_z.backward()
            D_G_z2 = output.data.mean()
            optimizerG.step()

            end_iter = time.time()
            
            #Print the info
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            
            #Save a grid with the pictures from the dataset, up until 64
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake,_ = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % epoch_interval == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [29]:
def train_our(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir,
                        model_option =1,binary = False, epoch_interval = 1,
                        D_steps = 1, G_steps = 1):
    use_gpu = tc.cuda.is_available()

    for epoch in range(num_epochs):
          
        for z in range(D_steps):
            if z > 3:
                raise ValueError('KEEP IT LOW!')
                
            for j, data in enumerate(dataloader, 0):
                start_iter = time.time()


                ############################
                # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
                # 1A - Train the detective network in the Real Dataset
                ###########################
                # train with real
                netD.zero_grad()
                real_cpu, _ = data
                batch_size = real_cpu.size(0)
                input.data.resize_(real_cpu.size()).copy_(real_cpu)
                label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

                output = netD(input)
                errD_real = criterion(output, label)
                errD_real.backward()

                #######################################################

                #######################################################
                # 1B - Train the detective network in the False Dataset
                #######################################################

                D_x = output.data.mean()
                # train with fake
                noise.data.resize_(batch_size, nz, 1, 1)
                if binary:
                    bernoulli_prob.resize_(noise.data.size())
                    noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
                else:
                    noise.data.normal_(0, 1)
                fake,z_prediction = netG(noise)
                label.data.fill_(fake_label)
                output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
                errD_fake = criterion(output, label)
                errD_fake.backward() # gradients for fake/real will be accumulated
                D_G_z1 = output.data.mean()
                errD = errD_real + errD_fake
                optimizerD.step() # .step() can be called once the gradients are computed

                #######################################################
        for z in range(G_steps):
            for i, data in enumerate(dataloader, 0):
                start_iter = time.time()
                # G_steps > D_steps (G_steps \geq D_steps)
                if z > 3:
                    raise ValueError('KEEP IT LOW!')
                #######################################################
                # (2) Update G network: maximize log(D(G(z)))
                #  Train the faker with de output from the Detective (but don't train the Detective)
                #############3#########################################
                netG.zero_grad()
                label.data.fill_(real_label) # fake labels are real for generator cost
                output = netD(fake)
                errG = criterion(output, label)
                errG.backward(retain_variables=True) # True if backward through the graph for the second time
                if model_option == 2: # with z predictor
                    errG_z = criterion_MSE(z_prediction, noise)
                    errG_z.backward()
                D_G_z2 = output.data.mean()
                optimizerG.step()

                end_iter = time.time()
                
                #Print the info
                print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                      % (epoch, num_epochs, i, len(dataloader),
                         errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            
            #Save a grid with the pictures from the dataset, up until 64
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake,_ = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % epoch_interval == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [ ]:
num_epochs = 100
d_labelSmooth = 0.2

train_both_networks(num_epochs, dataloader, netD_parallel,netG_parallel,d_labelSmooth, outputDir)

/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


[0/100][0/472] Loss_D: 1.9173 Loss_G: 4.7895 D(x): 0.3830 D(G(z)): 0.5337 / 0.0111 Elapsed 0.59 s
[0/100][1/472] Loss_D: 1.2354 Loss_G: 6.1327 D(x): 0.9514 D(G(z)): 0.3068 / 0.0031 Elapsed 0.06 s
[0/100][2/472] Loss_D: 0.8155 Loss_G: 7.0067 D(x): 0.9075 D(G(z)): 0.0802 / 0.0013 Elapsed 0.06 s
[0/100][3/472] Loss_D: 0.7080 Loss_G: 6.9739 D(x): 0.7305 D(G(z)): 0.0298 / 0.0015 Elapsed 0.06 s
[0/100][4/472] Loss_D: 0.8107 Loss_G: 5.8254 D(x): 0.5900 D(G(z)): 0.0362 / 0.0048 Elapsed 0.06 s
[0/100][5/472] Loss_D: 0.8067 Loss_G: 4.9768 D(x): 0.7065 D(G(z)): 0.0959 / 0.0103 Elapsed 0.06 s
[0/100][6/472] Loss_D: 0.8633 Loss_G: 5.3872 D(x): 0.8564 D(G(z)): 0.1714 / 0.0073 Elapsed 0.06 s
[0/100][7/472] Loss_D: 0.8169 Loss_G: 6.4374 D(x): 0.8786 D(G(z)): 0.1760 / 0.0026 Elapsed 0.06 s
[0/100][8/472] Loss_D: 0.7498 Loss_G: 7.6252 D(x): 0.7529 D(G(z)): 0.0549 / 0.0008 Elapsed 0.06 s
[0/100][9/472] Loss_D: 0.7381 Loss_G: 8.0523 D(x): 0.7006 D(G(z)): 0.0262 / 0.0007 Elapsed 0.06 s
[0/100][10/472] Loss

[0/100][85/472] Loss_D: 0.5464 Loss_G: 8.5713 D(x): 0.8131 D(G(z)): 0.0020 / 0.0002 Elapsed 0.06 s
[0/100][86/472] Loss_D: 0.6760 Loss_G: 6.7868 D(x): 0.6319 D(G(z)): 0.0024 / 0.0012 Elapsed 0.06 s
[0/100][87/472] Loss_D: 0.6430 Loss_G: 6.1328 D(x): 0.9073 D(G(z)): 0.0146 / 0.0023 Elapsed 0.06 s
[0/100][88/472] Loss_D: 0.6830 Loss_G: 6.9767 D(x): 0.8831 D(G(z)): 0.0322 / 0.0010 Elapsed 0.06 s
[0/100][89/472] Loss_D: 0.6116 Loss_G: 8.1103 D(x): 0.8146 D(G(z)): 0.0132 / 0.0003 Elapsed 0.06 s
[0/100][90/472] Loss_D: 0.6364 Loss_G: 8.2270 D(x): 0.7290 D(G(z)): 0.0054 / 0.0003 Elapsed 0.06 s
[0/100][91/472] Loss_D: 0.7213 Loss_G: 6.4014 D(x): 0.5978 D(G(z)): 0.0064 / 0.0018 Elapsed 0.06 s
[0/100][92/472] Loss_D: 0.6497 Loss_G: 5.9927 D(x): 0.7977 D(G(z)): 0.0483 / 0.0026 Elapsed 0.06 s
[0/100][93/472] Loss_D: 0.6727 Loss_G: 8.6965 D(x): 0.8520 D(G(z)): 0.1001 / 0.0002 Elapsed 0.06 s
[0/100][94/472] Loss_D: 0.6081 Loss_G: 9.9745 D(x): 0.7268 D(G(z)): 0.0080 / 0.0000 Elapsed 0.06 s
[0/100][95

[0/100][169/472] Loss_D: 0.5536 Loss_G: 5.1299 D(x): 0.8000 D(G(z)): 0.0133 / 0.0069 Elapsed 0.06 s
[0/100][170/472] Loss_D: 0.5576 Loss_G: 5.0628 D(x): 0.7939 D(G(z)): 0.0164 / 0.0076 Elapsed 0.06 s
[0/100][171/472] Loss_D: 0.5491 Loss_G: 5.2493 D(x): 0.7927 D(G(z)): 0.0137 / 0.0058 Elapsed 0.06 s
[0/100][172/472] Loss_D: 0.5604 Loss_G: 5.0048 D(x): 0.7992 D(G(z)): 0.0202 / 0.0077 Elapsed 0.06 s
[0/100][173/472] Loss_D: 0.5570 Loss_G: 5.2978 D(x): 0.8061 D(G(z)): 0.0172 / 0.0058 Elapsed 0.06 s
[0/100][174/472] Loss_D: 0.5475 Loss_G: 5.5032 D(x): 0.7931 D(G(z)): 0.0142 / 0.0046 Elapsed 0.06 s
[0/100][175/472] Loss_D: 0.5405 Loss_G: 5.8541 D(x): 0.7974 D(G(z)): 0.0106 / 0.0035 Elapsed 0.06 s
[0/100][176/472] Loss_D: 0.5730 Loss_G: 5.2660 D(x): 0.6884 D(G(z)): 0.0108 / 0.0062 Elapsed 0.06 s
[0/100][177/472] Loss_D: 0.5578 Loss_G: 4.6693 D(x): 0.7529 D(G(z)): 0.0205 / 0.0124 Elapsed 0.06 s
[0/100][178/472] Loss_D: 0.5984 Loss_G: 4.5838 D(x): 0.8462 D(G(z)): 0.0451 / 0.0134 Elapsed 0.06 s


[0/100][253/472] Loss_D: 0.5931 Loss_G: 4.6850 D(x): 0.7019 D(G(z)): 0.0222 / 0.0116 Elapsed 0.06 s
[0/100][254/472] Loss_D: 0.6239 Loss_G: 4.6753 D(x): 0.7005 D(G(z)): 0.0125 / 0.0114 Elapsed 0.06 s
[0/100][255/472] Loss_D: 0.6068 Loss_G: 3.9163 D(x): 0.7657 D(G(z)): 0.0282 / 0.0261 Elapsed 0.06 s
[0/100][256/472] Loss_D: 0.6487 Loss_G: 3.4186 D(x): 0.8185 D(G(z)): 0.0647 / 0.0423 Elapsed 0.06 s
[0/100][257/472] Loss_D: 0.6083 Loss_G: 4.0480 D(x): 0.8351 D(G(z)): 0.0568 / 0.0246 Elapsed 0.06 s
[0/100][258/472] Loss_D: 0.6000 Loss_G: 4.5988 D(x): 0.7855 D(G(z)): 0.0363 / 0.0152 Elapsed 0.06 s
[0/100][259/472] Loss_D: 0.6133 Loss_G: 4.4038 D(x): 0.7493 D(G(z)): 0.0442 / 0.0214 Elapsed 0.06 s
[0/100][260/472] Loss_D: 0.6245 Loss_G: 4.2786 D(x): 0.7124 D(G(z)): 0.0444 / 0.0251 Elapsed 0.06 s
[0/100][261/472] Loss_D: 0.6055 Loss_G: 4.0728 D(x): 0.7271 D(G(z)): 0.0504 / 0.0302 Elapsed 0.06 s
[0/100][262/472] Loss_D: 0.6775 Loss_G: 3.5804 D(x): 0.7038 D(G(z)): 0.0553 / 0.0397 Elapsed 0.06 s


...

[30/100][89/472] Loss_D: 0.6470 Loss_G: 4.4457 D(x): 0.6398 D(G(z)): 0.0125 / 0.0155 Elapsed 0.06 s
[30/100][90/472] Loss_D: 0.5768 Loss_G: 3.7019 D(x): 0.7672 D(G(z)): 0.0303 / 0.0372 Elapsed 0.06 s
[30/100][91/472] Loss_D: 0.6037 Loss_G: 3.4116 D(x): 0.7429 D(G(z)): 0.0368 / 0.0453 Elapsed 0.06 s
[30/100][92/472] Loss_D: 0.6068 Loss_G: 3.9518 D(x): 0.8247 D(G(z)): 0.0293 / 0.0278 Elapsed 0.06 s
[30/100][93/472] Loss_D: 0.6534 Loss_G: 3.6798 D(x): 0.8542 D(G(z)): 0.0737 / 0.0416 Elapsed 0.06 s
[30/100][94/472] Loss_D: 0.5985 Loss_G: 4.5774 D(x): 0.8789 D(G(z)): 0.0428 / 0.0157 Elapsed 0.06 s
[30/100][95/472] Loss_D: 0.6365 Loss_G: 4.0691 D(x): 0.6965 D(G(z)): 0.0455 / 0.0222 Elapsed 0.06 s
[30/100][96/472] Loss_D: 0.5614 Loss_G: 5.0117 D(x): 0.8103 D(G(z)): 0.0226 / 0.0105 Elapsed 0.06 s
[30/100][97/472] Loss_D: 0.6838 Loss_G: 4.6133 D(x): 0.5743 D(G(z)): 0.0120 / 0.0135 Elapsed 0.06 s
[30/100][98/472] Loss_D: 0.5764 Loss_G: 4.2072 D(x): 0.7866 D(G(z)): 0.0215 / 0.0228 Elapsed 0.06 s


[30/100][173/472] Loss_D: 0.5723 Loss_G: 5.3865 D(x): 0.7311 D(G(z)): 0.0100 / 0.0062 Elapsed 0.06 s
[30/100][174/472] Loss_D: 0.5814 Loss_G: 5.0167 D(x): 0.7786 D(G(z)): 0.0137 / 0.0092 Elapsed 0.06 s
[30/100][175/472] Loss_D: 0.5763 Loss_G: 5.0232 D(x): 0.7995 D(G(z)): 0.0149 / 0.0098 Elapsed 0.06 s
[30/100][176/472] Loss_D: 0.5714 Loss_G: 5.1327 D(x): 0.7512 D(G(z)): 0.0101 / 0.0080 Elapsed 0.06 s
[30/100][177/472] Loss_D: 0.5808 Loss_G: 4.8562 D(x): 0.7814 D(G(z)): 0.0141 / 0.0114 Elapsed 0.06 s
[30/100][178/472] Loss_D: 0.5797 Loss_G: 4.4667 D(x): 0.7647 D(G(z)): 0.0192 / 0.0169 Elapsed 0.06 s
[30/100][179/472] Loss_D: 0.6790 Loss_G: 3.3899 D(x): 0.6182 D(G(z)): 0.0290 / 0.0457 Elapsed 0.06 s
[30/100][180/472] Loss_D: 0.6019 Loss_G: 3.5911 D(x): 0.8427 D(G(z)): 0.0326 / 0.0356 Elapsed 0.06 s
[30/100][181/472] Loss_D: 0.6051 Loss_G: 3.5792 D(x): 0.8321 D(G(z)): 0.0457 / 0.0364 Elapsed 0.06 s
[30/100][182/472] Loss_D: 0.6864 Loss_G: 3.9499 D(x): 0.9202 D(G(z)): 0.0617 / 0.0248 Elaps

[30/100][257/472] Loss_D: 0.6626 Loss_G: 2.5286 D(x): 0.6694 D(G(z)): 0.0625 / 0.1029 Elapsed 0.06 s
[30/100][258/472] Loss_D: 0.7105 Loss_G: 3.1651 D(x): 0.8742 D(G(z)): 0.1061 / 0.0681 Elapsed 0.06 s
[30/100][259/472] Loss_D: 0.7220 Loss_G: 3.4104 D(x): 0.8003 D(G(z)): 0.1453 / 0.0497 Elapsed 0.06 s
[30/100][260/472] Loss_D: 0.5974 Loss_G: 5.0178 D(x): 0.8458 D(G(z)): 0.0422 / 0.0098 Elapsed 0.06 s
[30/100][261/472] Loss_D: 0.6524 Loss_G: 5.3688 D(x): 0.6261 D(G(z)): 0.0133 / 0.0074 Elapsed 0.06 s
[30/100][262/472] Loss_D: 0.5694 Loss_G: 5.3055 D(x): 0.7703 D(G(z)): 0.0139 / 0.0089 Elapsed 0.06 s
[30/100][263/472] Loss_D: 0.5637 Loss_G: 5.3012 D(x): 0.7952 D(G(z)): 0.0121 / 0.0076 Elapsed 0.06 s
[30/100][264/472] Loss_D: 0.8532 Loss_G: 4.1233 D(x): 0.4677 D(G(z)): 0.0079 / 0.0222 Elapsed 0.06 s
[30/100][265/472] Loss_D: 0.6288 Loss_G: 2.6670 D(x): 0.6925 D(G(z)): 0.0313 / 0.0966 Elapsed 0.06 s
[30/100][266/472] Loss_D: 0.8607 Loss_G: 2.4679 D(x): 0.9349 D(G(z)): 0.1615 / 0.1193 Elaps

[30/100][341/472] Loss_D: 0.6448 Loss_G: 4.3343 D(x): 0.7896 D(G(z)): 0.0737 / 0.0184 Elapsed 0.06 s
[30/100][342/472] Loss_D: 0.7158 Loss_G: 4.3604 D(x): 0.5772 D(G(z)): 0.0313 / 0.0198 Elapsed 0.06 s
[30/100][343/472] Loss_D: 0.7505 Loss_G: 3.0556 D(x): 0.5455 D(G(z)): 0.0399 / 0.0668 Elapsed 0.06 s
[30/100][344/472] Loss_D: 0.6340 Loss_G: 2.6852 D(x): 0.7397 D(G(z)): 0.0690 / 0.0889 Elapsed 0.06 s
[30/100][345/472] Loss_D: 0.8123 Loss_G: 2.7625 D(x): 0.8416 D(G(z)): 0.2051 / 0.0901 Elapsed 0.06 s
[30/100][346/472] Loss_D: 0.6592 Loss_G: 4.6312 D(x): 0.8774 D(G(z)): 0.0830 / 0.0194 Elapsed 0.06 s
[30/100][347/472] Loss_D: 1.1097 Loss_G: 3.6926 D(x): 0.3511 D(G(z)): 0.0311 / 0.0383 Elapsed 0.06 s
[30/100][348/472] Loss_D: 0.6856 Loss_G: 2.8177 D(x): 0.6588 D(G(z)): 0.0605 / 0.0819 Elapsed 0.06 s
[30/100][349/472] Loss_D: 0.6856 Loss_G: 3.0565 D(x): 0.8632 D(G(z)): 0.1028 / 0.0647 Elapsed 0.06 s
[30/100][350/472] Loss_D: 0.6977 Loss_G: 3.7173 D(x): 0.8173 D(G(z)): 0.1208 / 0.0378 Elaps

[30/100][425/472] Loss_D: 0.5738 Loss_G: 3.5578 D(x): 0.7842 D(G(z)): 0.0306 / 0.0448 Elapsed 0.06 s
[30/100][426/472] Loss_D: 0.7092 Loss_G: 3.1487 D(x): 0.9083 D(G(z)): 0.0934 / 0.0637 Elapsed 0.06 s
[30/100][427/472] Loss_D: 0.7374 Loss_G: 4.2371 D(x): 0.9281 D(G(z)): 0.0724 / 0.0242 Elapsed 0.06 s
[30/100][428/472] Loss_D: 0.5817 Loss_G: 5.2602 D(x): 0.8546 D(G(z)): 0.0302 / 0.0080 Elapsed 0.06 s
[30/100][429/472] Loss_D: 0.5650 Loss_G: 6.0710 D(x): 0.7721 D(G(z)): 0.0123 / 0.0038 Elapsed 0.06 s
[30/100][430/472] Loss_D: 0.5468 Loss_G: 6.8024 D(x): 0.7343 D(G(z)): 0.0040 / 0.0018 Elapsed 0.06 s
[30/100][431/472] Loss_D: 0.6848 Loss_G: 5.3732 D(x): 0.5690 D(G(z)): 0.0065 / 0.0075 Elapsed 0.06 s
[30/100][432/472] Loss_D: 0.7208 Loss_G: 4.4516 D(x): 0.5592 D(G(z)): 0.0069 / 0.0176 Elapsed 0.06 s
[30/100][433/472] Loss_D: 0.6074 Loss_G: 2.7656 D(x): 0.7263 D(G(z)): 0.0411 / 0.0869 Elapsed 0.06 s
[30/100][434/472] Loss_D: 0.6913 Loss_G: 2.7117 D(x): 0.8753 D(G(z)): 0.0872 / 0.0874 Elaps